In [5]:
using Plots

# Constants
ρ = 1.225     # Air density (kg/m^3)
A1 = 144.0      # Area of wider section (m^2)
A2 = 30.0      # Area of narrower section (m^2)
g = 9.81      # Acceleration due to gravity (m/s^2)

# Function to calculate pressure using Bernoulli's equation
function bernoulli(p1, v1, p2, v2)
    return p1 + 0.5 * ρ * v1^2 - (p2 + 0.5 * ρ * v2^2)
end

# Calculate pressure at the North wind tunnel (narrow section) using Bernoulli's equation
function venturi_effect(p1, v1, A1, A2)
    v2 = (A1 / A2) * v1
    p2 = p1 + 0.5 * ρ * (v1^2 - v2^2)
    return p2
end

# Simulate pressure change along the length of the tunnel
function simulate_venturi_effect(p1, v1, A1, A2, length)
    Δx = 0.01   # Step size
    x_values = collect(0:Δx:length)
    p_values = [p1]
    for x in x_values[2:end]
        p2 = venturi_effect(p1, v1, A1, A2)
        push!(p_values, p2)
    end
    return x_values, p_values
end

# Function to calculate the force exerted by the wind on the undergrad
function wind_force(p, A)
    return p * A
end

# Function to calculate the torque required to topple the undergrad
function torque_required(w, h)
    return w * h / 2   # Assuming the center of mass is at the center of the undergrad
end

# Function to simulate whether the undergrad will be knocked over
function simulate_knockover(p_values, A_object, weight_object, height_object)
    torque = torque_required(weight_object, height_object)
    for p in p_values
        force = wind_force(p, A_object)
        if force * height_object > torque
            return true  # Undergrad will be knocked over
        end
    end
    return false  # Undergrad will not be knocked over
end

# Function to find the initial velocity required to knock the object over
function find_required_velocity(object_area, object_weight, object_height)
    v1 = 1.0  # Initial velocity
    while true
        # Simulate pressure change
        _, p_values = simulate_venturi_effect(1.0, v1, A1, A2, 10.0)
        # Simulate whether the undergrad will be knocked over
        knocked_over = simulate_knockover(p_values, object_area, object_weight, object_height)
        if knocked_over
            return v1
        end
        v1 += 1.0  # Increment velocity
    end
end

# Simulation parameters for undergrad
object_area = 0.85      # Area of the undergrad (m^2)
object_weight = 600.0    # Weight of the undergrad (N)
object_height = 1.2    # Height of the undergrad (m)

# Find the initial velocity required to knock the object over
required_velocity = find_required_velocity(object_area, object_weight, object_height)
println("The initial velocity needed to knock the undergrad over is $required_velocity m/s.")